        06102024
        Remove redunant sequences (15%/85%) in PG9. 
        

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os.path

In [2]:
# Step 1: load results. 
def load_results(which_search): 
    '''
    input: all_log.txt 
    output: parsed all_log: donor_names, cdrh3_seqs_by_donor
    '''
    logfile_path = f"all_log.txt"  #print(f">Loading in search results for: {which_search}")
    
    # STEP 1: open and read the log file which contains the search results:
    with open(logfile_path, "r") as fd: # import the results file 
        logfile = fd.read().splitlines() # read in the results
        logfile_split = [] 
        for line in logfile:
            logfile_split.append(line.split()) # save each line to the list logfile_split
    fd.close()
    
    # STEP 2: save out the donor names and cdrh3 seqs: (full seqs not needed right now)
    donor_names = []
    cdrh3_seqs  = []
    full_seqs   = []
    dgene = []
    for line in range(len(logfile_split)): # for each line 
        for entry in range(len(logfile_split[line])): # for each entry in each line
            if logfile_split[line][entry].rfind("IGHV") == 0: # find which entry the Vgene is at 
                dgene.append(logfile_split[line][entry+1]) # save the d gene 
                if 'csv' in logfile_split[line][entry-1]: # if a .csv file is right before the Vgene, 
                    donor_names.append("no_donor_name_found") # then this line has no donor name
                else: # if the .csv line is not right before the Vgene, 
                    donor_names.append(logfile_split[line][entry-1]) # then save the reported donor name  
            if logfile_split[line][entry].isdigit() == True:
                if int(logfile_split[line][entry]) == len(logfile_split[line][entry-1]):
                    cdrh3_seqs.append(logfile_split[line][entry-1])     
                
        full_seqs.append(logfile_split[line][-2])
    
    
    #print(len(dgene), len(cdrh3_seqs), len(donor_names))
    
    updated_donor_names = []
    updated_cdrh3_seqs = []
    updated_dgene = []
    updated_full_seqs = [] 
    for i in range(len(donor_names)):
        if donor_names[i] != 'no_donor_name_found' and donor_names[i] != 'no':
            updated_donor_names.append(donor_names[i])
            updated_cdrh3_seqs.append(cdrh3_seqs[i])
            updated_dgene.append(dgene[i])
            updated_full_seqs.append(full_seqs[i])
            
    # STEP 3: save / group sequences according to donor names
    unique_donor_names = np.unique(updated_donor_names)
    cdrh3_seqs_by_donor = []
    full_seqs_by_donor = []
    for i in range(len(unique_donor_names)):
        tmp_cdrh3 = []
        tmp_fullseq = []
        for j in range(len(donor_names)):
            tmp_tmp_cdrh3 = []
            tmp_tmp_fullseq = []
            if donor_names[j] == unique_donor_names[i] and donor_names[j] != 'no' and donor_names[j] != 'no_donor_name_found':
                tmp_tmp_cdrh3.append(cdrh3_seqs[j])
                tmp_tmp_fullseq.append(full_seqs[j])
            tmp_cdrh3.append(tmp_tmp_cdrh3)
            tmp_fullseq.append(tmp_tmp_fullseq)
        cdrh3_seqs_by_donor.append(np.concatenate(tmp_cdrh3))
        full_seqs_by_donor.append(np.concatenate(tmp_fullseq))
    return updated_donor_names, cdrh3_seqs_by_donor, full_seqs_by_donor, updated_cdrh3_seqs, updated_dgene

In [3]:
pwd

'/Users/ssolieva/Desktop/Kulp_lab/projects/OAS_database_searches/PG9/PG9_noVgene_cdrh3_anylen_motif_4'

In [4]:
which_search = 'PG9_noVgene_cdrh3_anylen_motif_4'
donor_names, cdrh3_seqs_by_donor, full_seqs_by_donor, cdrh3_seqs, dgenes = load_results(which_search) 

In [5]:
print(len(cdrh3_seqs_by_donor), len(full_seqs_by_donor), len(donor_names), len(cdrh3_seqs))

152 152 292238 292238


In [6]:
unique_cdrh3_seqs_by_donor_ind = []
for i in range(len(cdrh3_seqs_by_donor)):
    unique_cdrh3_seqs_by_donor_ind.append(np.unique(cdrh3_seqs_by_donor[i], return_index=True))

unique_full_seq_by_donor = []
for i in range(len(full_seqs_by_donor)):
    unique_full_seq_by_donor0 = []
    for j in range(len(unique_cdrh3_seqs_by_donor_ind[i][1])):
        #print(i,j, unique_cdrh3_seqs_by_donor_ind[i][1][j])
        n = unique_cdrh3_seqs_by_donor_ind[i][1][j]
        unique_full_seq_by_donor0.append(full_seqs_by_donor[i][n])
    unique_full_seq_by_donor.append(unique_full_seq_by_donor0)

def diff_letters(a,b):
    '''reports the number of different letters (residues)'''
    return sum ( a[i] != b[i] for i in range(len(a)) )

    
def remove_almost_duplicates(dataset_cdrh3, dataset_full_seq):
    '''
    data = unique_cdrh3_seqs_by_donor[0]
    '''
    dataset = dataset_cdrh3
    
    different_seqs_final      = [] # final cdrh3 sequences
    different_seqs_final_full = [] # final full sequences
    
    # get unique lengths:
    unique_lengths = []
    for seq in range(len(dataset)):
        unique_lengths.append(len(dataset[seq]))
    unique_lengths = np.unique(unique_lengths)
    #print(unique_lengths)
    
    # group data into unique length groups:
    unique_length_groups = []
    unique_length_groups_full = []
    for length in range(len(unique_lengths)):
        unique_length_groups_0 = []
        unique_length_groups_0_full = []
        for seq in range(len(dataset)):
            unique_length_groups_00 = []
            unique_length_groups_00_full = []
            if len(dataset[seq]) == unique_lengths[length]:
                unique_length_groups_00.append(dataset[seq])
                unique_length_groups_00_full.append(dataset_full_seq[seq])
            unique_length_groups_0.append(unique_length_groups_00)
            unique_length_groups_0_full.append(unique_length_groups_00_full)
        unique_length_groups.append(np.concatenate(unique_length_groups_0))
        unique_length_groups_full.append(np.concatenate(unique_length_groups_0_full))
    #print(d, len(unique_length_groups), unique_lengths)
    
    # for each unique length group, get rid of duplicates and almost duplicates
    for group in range(len(unique_length_groups)):
        if len(unique_length_groups[group]) == 1: # if there's only 1 sequence, then save that sequence
            #print(len(unique_length_groups[group]), unique_length_groups[group][0])
            different_seqs_final.append(unique_length_groups[group][0])
            different_seqs_final_full.append(unique_length_groups_full[group][0])
        
        all_seqs_updated = []
        all_seqs_updated_full = [] # new line
        
        if len(unique_length_groups[group]) > 1: # if more than 1 sequence, 
            all_seqs = unique_length_groups[group]
            all_seqs_full = unique_length_groups_full[group] #unique_length_groups[group] # new line
            
            a = unique_length_groups[group]
            b = unique_length_groups[group]
            pairs = [(i, j) for i in a for j in b if i < j] # make pairwise pairs of the sequences
            #print('number of pairs:', len(pairs))
            
            for pair in range(len(pairs)):
                n_diff = diff_letters(pairs[pair][0], pairs[pair][1])
                if n_diff <= (int(len(pairs[pair][0])*0.15)):                 # 15% / 85%
                   # print(n_diff, pairs[pair][0], pairs[pair][1])
                    result = np.where(all_seqs==f'{pairs[pair][1]}')[0]
                    all_seqs = np.delete(all_seqs, result) 
                    all_seqs_full =  np.delete(all_seqs_full, result) 
            for al in range(len(all_seqs)):
                all_seqs_updated.append(all_seqs[al])
                all_seqs_updated_full.append(all_seqs_full[al])
            
        #print('all_seqs_updated', all_seqs_updated[0],  all_seqs_updated_full[0])
        #print(all_seqs_updated, len(all_seqs_updated))
        if len(all_seqs_updated) != 0:
            for ds in range(len(all_seqs_updated)):
                different_seqs_final.append(all_seqs_updated[ds])
                different_seqs_final_full.append(all_seqs_updated_full[ds])
                
    return different_seqs_final, different_seqs_final_full

In [7]:
def remove_exact_cdrh3_duplicates_within_donors(full_seqs_by_donor, cdrh3_seqs_by_donor):
    print('full sequences')
    unique_full_seqs_by_donor = []
    for i in range(len(full_seqs_by_donor)):
        unique_full_seqs_by_donor.append(np.unique(full_seqs_by_donor[i]))
    print(f'\tnumber of duplicate full sequences: {len(np.concatenate(full_seqs_by_donor)) - len(np.concatenate(unique_full_seqs_by_donor))}')
    print(f'\tn_seq after full sequence duplicates within each donor removed: {len(np.concatenate(unique_full_seqs_by_donor))}')
    print('cdrh3 sequences')
    unique_cdrh3_seqs_by_donor = []
    for i in range(len(cdrh3_seqs_by_donor)):
        unique_cdrh3_seqs_by_donor.append(np.unique(cdrh3_seqs_by_donor[i]))
    print(f'\tnumber of duplicate CDRH3 sequences: {len(np.concatenate(full_seqs_by_donor))- len(np.concatenate(unique_cdrh3_seqs_by_donor))}')
    print(f'\tnumber of sequences after removing exact CDRH3 duplicate sequences: {len(np.concatenate(unique_cdrh3_seqs_by_donor))}')
    return unique_full_seqs_by_donor, unique_cdrh3_seqs_by_donor

In [8]:
unique_full_seqs_by_donor, unique_cdrh3_seqs_by_donor = remove_exact_cdrh3_duplicates_within_donors(full_seqs_by_donor, cdrh3_seqs_by_donor)

full sequences
	number of duplicate full sequences: 26668
	n_seq after full sequence duplicates within each donor removed: 265570
cdrh3 sequences
	number of duplicate CDRH3 sequences: 209770
	number of sequences after removing exact CDRH3 duplicate sequences: 82468


In [9]:
different_seqs_final_all = []
different_seqs_final_full_all = []

for donor in range(len(unique_cdrh3_seqs_by_donor)):
    different_seqs_final,different_seqs_final_full = remove_almost_duplicates(unique_cdrh3_seqs_by_donor[donor], unique_full_seq_by_donor[donor])
    different_seqs_final_all.append(different_seqs_final)
    different_seqs_final_full_all.append(different_seqs_final_full)

In [10]:
different_seqs_final_full_all_concat = np.concatenate(different_seqs_final_full_all)
np.save("PG9_noVgene_cdrh3_anylen_motif_4__full_seqs.npy", different_seqs_final_full_all_concat)

In [14]:
len(different_seqs_final_full_all_concat)

38427

In [15]:
pwd

'/Users/ssolieva/Desktop/Kulp_lab/projects/OAS_database_searches/PG9/PG9_noVgene_cdrh3_anylen_motif_4'

In [16]:
len(different_seqs_final_full_all_concat)/20

1921.35